In [1]:
from PIL import Image
import numpy as np
from chainer import Variable, FunctionSet, optimizers,serializers,cuda
import chainer.functions  as F
np.random.seed(0)
import chainer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math

In [2]:
def load_image(filepath):
    x_train = [np.array(Image.open(filepath).resize((im_size,im_size)))]
    x_train=np.array(x_train)
    x_train=x_train.astype(np.int32)/255.0
    x_train=np.transpose(x_train,(0,3,1,2))
    x_train=x_train.astype(np.float32)
    return x_train

def show_img(path):
    image = mpimg.imread(path)
    plt.imshow(image)
    plt.show()

#test
#data = load_image('384000/384048.jpeg')
#print(data.shape)

In [3]:
#gls np array of ground-truth labels
#yls np array of predict labels
def labels_entropy(gls, yls):
    assert(len(gls) == len(yls))
    
    '''
    e = 0
    for i in range(len(gls)):
        c = entropy(gls[i], yls[i])
        if(math.isnan(c)):
            print(gls[i], yls[i])
        e += c
    '''
        
    e = np.sum(entropy(gls, yls))
    return e

#print(labels_entropy(np.array([1,1]), np.array([0.9,0.9])))

In [4]:
# g:ground-truth label t (0 or 1)
# y:predict label t (0 or 1)
def entropy(g, y):    
    e = -(g * np.log(y) + (1 - g) * np.log(1 - y))
    
    return e

#print(entropy(1, 0.01))

nan


/Users/serv8800/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/Users/serv8800/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [5]:
def load_labels():
    labelidx={}
    f=open("corel5k_words.txt")
    line=f.read()
    f.close()
    line=line.split('\n')
    del line[260]
    for i in range(len(line)):
        labelidx[line[i]] = i
    
    return labelidx

labelidx = load_labels()

In [6]:
def load_gt_labels():
    gt_labels = []
    with open("ground_truth_labels.txt") as f:
        for line in f:
            t = line.strip('\n')
        
            labels = np.zeros(260)
            for label in (t.split('|')):
                idx = labelidx[label]
                labels[idx] = 1
            
            gt_labels.append(labels)
    return gt_labels

gt_labels = load_gt_labels()

In [7]:
im_size=127

threshold = 0.2

model_path = {
        'without' : 'model/model_wd_0.001_adam_0.0001.model',
        'word2vec' : 'model/hw2v_model_0.0000100000_1.00_1000.model', }

without_model = FunctionSet(conv1=F.Convolution2D(3,  96, 11, stride=4),
                    bn1=F.BatchNormalization(96),
                    conv2=F.Convolution2D(96, 256,  5, pad=2),
                    bn2=F.BatchNormalization(256),
                    conv3=F.Convolution2D(256, 384,  3, pad=1),
                    conv4=F.Convolution2D(384, 384,  3, pad=1),
                    conv5=F.Convolution2D(384, 256,  3, pad=1),
                    fc6=F.Linear(2304,1024),
                    fc7=F.Linear(1024, 260))

serializers.load_npz(model_path['without'], without_model)

word2vec_model = FunctionSet(conv1=F.Convolution2D(3,  96, 11, stride=4),
                    bn1=F.BatchNormalization(96),
                    conv2=F.Convolution2D(96, 256,  5, pad=2),
                    bn2=F.BatchNormalization(256),
                    conv3=F.Convolution2D(256, 384,  3, pad=1),
                    conv4=F.Convolution2D(384, 384,  3, pad=1),
                    conv5=F.Convolution2D(384, 256,  3, pad=1),
                    fc6=F.Linear(2304,1024),
                    fc7=F.Linear(1024, 260))

serializers.load_npz(model_path['word2vec'], word2vec_model)

In [8]:
def predict(model, x_data):
    x = Variable(x_data)
    h=F.max_pooling_2d(F.relu(F.local_response_normalization(model.conv1(x))),3,stride=2)
    h=F.max_pooling_2d(F.relu(F.local_response_normalization(model.conv2(h))),3,stride=2)
    h=F.relu(model.conv3(h))
    h=F.relu(model.conv4(h))
    h=F.max_pooling_2d(F.relu(model.conv5(h)),3,stride=2)
    h=F.relu(model.fc6(h))
    y = model.fc7(h)

    y_f=F.sigmoid(y)
    return y_f

def predict_labels(model, image_path):
    labels = np.zeros(260)
    
    xdata = load_image(image_path)

    y_f = predict(model, xdata)

    label_prob = y_f.data[0,:]

    limit = 0
    idxsort = label_prob.argsort()
    for i in range(len(idxsort)):
        i = -i - 1
        prob = label_prob[idxsort[i]]
        
        idx = idxsort[i]
        labels[idx] = prob
          
    return labels

In [9]:
#labels = predict_labels(word2vec_model, "1000/1001.jpeg")
#e = labels_entropy(gt_labels[0], labels)
#print(e)

In [10]:
def gen_test_co():
    test_co = {}
    test_idx = {}
    idx = 0
    with open('corel5k_test_list.txt') as f:
        for line in f:
            
            test_idx[line.strip('\n')] = idx
            idx = idx + 1
            
            t = line.strip('\n').split('/')
            if not t[0] in test_co:
                test_co[t[0]] = []
            test_co[t[0]].append(t[1])
            
    return test_co, test_idx

test_co, test_idx = gen_test_co()

In [11]:
idx = 2
path = 'test_set/pic_%d.jpeg' % (idx + 1)

labels = predict_labels(without_model, path)

#print(labels)

e = labels_entropy(gt_labels[idx], labels)

#print(e)

In [12]:
test_co, test_idx = gen_test_co()

def calc_model_entropy(model):
    total_entropy = 0
    for idx in range(100):
        #print("gen idx:", idx)

        path = 'test_set/pic_%d.jpeg' % (idx + 1)

        labels = predict_labels(model, path)

        e = labels_entropy(gt_labels[idx], labels)
        
        #print(e)

        total_entropy += e
            
    return total_entropy

#print(calc_model_entropy(without_model))

In [13]:
print(calc_model_entropy(without_model))
print(calc_model_entropy(word2vec_model))

3718.19528571
3517.81525553
